# Matching the WEST ICRH Antenna

In this notebook we investigate the various method to match a WEST ICRH antenna. By matching the antenna we mean to find 4 capacitances values $C_1$, $C_2$, $C_3$ and $C_4$ in order for the antenna to be operated at a given frequency $f_0$. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import skrf as rf
import numpy as np
import sys
from tqdm.notebook import tqdm

sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

In [ ]:
rf.stylely()

In [ ]:
from west_ic_antenna import WestIcrhAntenna

## Matching Each Sides Separately
Here, each side of the antenna is matched separatly, which leads to two set of capacitances $(C_1, C_2)$ and $(C_3,C_4)$.

In the following example, both sides of the antenna are matched at the frequency $f_0$, keeping opposite side unmatched (C=150pF):

In [ ]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

In [ ]:
C_match_left = ant.match_one_side(f_match=f0, side='left', solution_number=1)
C_match_right = ant.match_one_side(f_match=f0, side='right', solution_number=1)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

Let's have a look to the RF reflection coefficient of each sides:

In [ ]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))

In reality, the precision at which one can tune the capacitance is not better than 1/100 pF, so one have to consider rounding optimal solutions to such precision :


In [ ]:
C_match_left = ant.match_one_side(f_match=f0, side='left', solution_number=1, decimals=2)
C_match_right = ant.match_one_side(f_match=f0, side='right', solution_number=1, decimals=2)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

Note that the performances are slightly degraded, but, it's real life! 

In [ ]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))
ax.set_ylabel('$|S_{ii}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{11}$', '$S_{22}$'))

## Frequency Shift for Dipole Excitation
The coupling between antenna's sides requires shifting the frequency with respect to the matching frequency used for each side separately.

Thus, if the optimal capacitor set for both side are of the same kind (i.e. either both C_top>C_bot or both C_top<C_bot), then dipole excitation (phase $(0,\pi)$) requires to shift the frequency to higher frequency to operate the antenna in a optimal conditions:

In [ ]:
# dipole excitation
power = [1, 1]
phase = [0, rf.pi]
# combine both separate solutions
C_match = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_match)
# finding the optimum frequency
idx_f_opt = np.argmin(np.abs(s_act[:,0]))
f_opt = freq.f[idx_f_opt]
delta_f = f_opt - f0
print(f'Optimum frequency is f_opt={f_opt/1e6} MHz --> {delta_f/1e6} MHz shift' )

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.axvline(f_opt/1e6, ls='--', color='k')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Dipole excitation')

Monopole excitation (phase $(0,0)$) at the contrary requires shifting the operating frequency toward lower frequencies:

In [ ]:
# monopole excitation
power = [1, 1]
phase = [0, 0]
# combine both separate solutions
C_match = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_match)
# finding the optimum frequency
idx_f_opt = np.argmin(np.abs(s_act[:,0]))
f_opt = freq.f[idx_f_opt]
delta_f = f_opt - f0
print(f'Optimum frequency is f_opt={f_opt/1e6} MHz --> {delta_f/1e6} MHz shift' )

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.axvline(f_opt/1e6, ls='--', color='k')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Monopole excitation')

## Matching Both Sides at the same time
It is also possible to optimize the antenna directly for the target frequency and for a given excitation. Matching both sides at the same time in fact matches each side separately, then find the optimum points using these solutions as starting point to help the convergence of the optimization.

Two methods are available to match both sides of the antenna: `match_both_sides` which use the NumPy optimisation routines, and `match_both_sides_iterative` which uses the feedback matching algorithm (see [matching_automatic.ipynb](matching_automatic.ipynb) for more details). The primer is more robust in finding a solution, eventually at the price of a longer calculation. The later method can indeed be "lost" if the ideal solution is not close enough (or if the optimal region is too narrow, like under vacuum).

In [ ]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

# antenna excitation to match for
power = [1, 1]  # W
phase = [0, np.pi]  # dipole

ant.DEBUG=True  # display additional messages
# Providing an initial guess C0 skip the search on both sides separately
C0 = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
C_opt_vacuum_dipole = ant.match_both_sides(f_match=f0, power=power, phase=phase, C0=C0, decimals=2)

In [ ]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_opt_vacuum_dipole)

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))

# Manual Matching on Plasma
When the antenna is facing the plasma, the coupling resistance increases and the antenna matching is affected.

If the operator keeps the matchpoint obtained on vacuum (in dipole for example), the antenna will have the following behaviour:

In [ ]:
# changing the front-face of the antenna to a plasma case
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
front_face_plasma = WestIcrhAntenna.interpolate_front_face(Rc=1, source='TOPICA-L-mode')
ant = WestIcrhAntenna(frequency=freq, front_face=front_face_plasma)  

# looking to the active s parameters in dipole:
powers = [1 ,1]
phases = [0, np.pi] 
s_act = ant.s_act(powers, phases , Cs=C_opt_vacuum_dipole)

fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Reflection on plasma using vacuum matching setpoint')

Thus, it is necessary to re-adapt the capacitors to improve the matching of the antenna on plasma:

In [ ]:
C_match_plasma = ant.match_both_sides(f0, power=powers, phase=phases, C0=C_opt_vacuum_dipole)

In [ ]:
s_act = ant.s_act(powers, phases , Cs=C_match_plasma)

fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Reflection on plasma after rematching')

If we compare the differences between capacitances setpoint between vacuum and plasma cases, we see that the difference between the values go like this:
* top capacitances are increased 
* bottom capacitances are decreased

Thus, the "distance" between top and bottom capacitance is increased. 

In [ ]:
np.array(C_match_plasma) - np.array(C_vacuum)

The capacitance shift to apply with respect to the vacuum setpoint depends of the coupling resistance of the plasma.

We can use this property to deduce the set of capacitors to use during plasma operation. Below we generate a series of various plasma loading cases, of increasing coupling resistance $R_c$. For each case we search for and we store the setpoints.

In [ ]:
C_matchs = []

# note: on plasma loads, the iterative method is faster
# important : increase the gap between capacitances start point for the algorith to converge for high Rc cases
C0 = np.array(C_opt_vacuum_dipole) + np.array([+5, -5, +5, -5]) 

Rcs1 = np.linspace(0.39, 1.7, 10)  # TOPICA "H-mode" (low coupling)
for Rc in tqdm(Rcs1):
    _plasma = WestIcrhAntenna.interpolate_front_face(Rc=Rc, source='TOPICA-H-mode')
    _ant = WestIcrhAntenna(frequency=freq, front_face=_plasma)
    _C_match = _ant.match_both_sides_iterative(f0, power=powers, phase=phases, C0=C_opt_vacuum_dipole)
    C_matchs.append(_C_match)


Rcs2 = np.linspace(1, 2.9, 10)  # TOPICA "L-mode" (higher coupling)
for Rc in tqdm(Rcs2):
    _plasma = WestIcrhAntenna.interpolate_front_face(Rc=Rc, source='TOPICA-L-mode')
    _ant = WestIcrhAntenna(frequency=freq, front_face=_plasma)
    _C_match = _ant.match_both_sides_iterative(f0, power=powers, phase=phases, C0=C0)
    C_matchs.append(_C_match)

Let's plot the average capacitance shift to apply versus the coupling resistance of the plasma loading scenario:

In [ ]:
Rcs3 = np.linspace(0.3, 1.6, 10)

In [ ]:
diff_C = np.array(C_matchs) - np.array(C_vacuum)

In [ ]:
Rcs=np.concatenate([Rcs1, Rcs2])
fig, ax = plt.subplots()
ax.plot(Rcs, diff_C, 'o')
ax.set_xlabel('Coupling Resistance $R_c$ [Ohm]')
ax.set_ylabel('Capacitance Shift $\Delta C$ [pF]')
ax.set_title('Capacitance Shift to add from Vacuum Match Points (55 MHz)')
ax.legend(['Left Top', 'Left Bot', 'Right Top', 'Right Bot'])

In [ ]:
Cs=list(np.array(C_opt_vacuum_dipole) + np.array([+3, -3, +3, -3]))

In [ ]:
np.argmin(np.abs(ant.frequency.f - 55e6))

In [ ]:
C_left, C_right, err = ant.capacitor_predictor(powers, phases, Cs=Cs)
Cs=[*C_left[500], *C_right[500]]
print(Cs)

In [ ]:
s_act = ant.s_act(powers, phases , Cs=Cs)

fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Reflection on plasma after rematching')

This plot should be benchmark against plasma measurements.

In [ ]:
from IPython.core.display import HTML
def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter
    Pass the file path to the CSS file
    """
    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)

_set_css_style('custom.css')